# 1. Setup

1.1 Importing Libraries

In [11]:
import cv2
import os
import random
import numpy as np 
from PIL import Image
from matplotlib import pyplot as plt
import uuid

In [12]:
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

1.2 Set GPU Growth

In [13]:
# grabs all GPUs
gpus = tf.config.experimental.list_physical_devices("GPU")
# sets memory growth for each GPU
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

1.3 Create Folder Structures

In [14]:
POS_PATH = os.path.join("data", "positive")
NEG_PATH = os.path.join("data", "negative")
ANC_PATH = os.path.join("data", "anchor")

In [5]:
# make the directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

# 2. Collect Positives and Anchors

2.1 Untar Labelled Faces in the Wild Dataset

In [15]:
# http://vis-www.cs.umass.edu/lfw/

In [ ]:
# uncompress Tar GZ Labelled Faces in the Wild Dataset
!tar -xf lfw.tgz

In [ ]:
# move LFW Images to the following repository data/negative
for directory in os.listdir("lfw"):
    for file in os.listdir(os.path.join("lfw", directory)):
        EX_PATH = os.path.join("lfw", directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)

2.2 Collect Postitive and Anchor Images

In [16]:
# import uuid (universally unique identifiers) library to generate unique image name 
import uuid

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(3, 250)
cap.set(4, 250)

while True:
    ret, frame = cap.read() # reads each frame one by one
    
    # create positives
    if cv2.waitKey(1) & 0xFF == ord("p"):
        imname = os.path.join(POS_PATH, "{}.jpg".format(uuid.uuid1()))
        cv2.imwrite(imname, frame)
    # create anchors
    if cv2.waitKey(1) & 0xFF == ord("a"):
        imname = os.path.join(ANC_PATH, "{}.jpg".format(uuid.uuid1()))
        cv2.imwrite(imname, frame)



    cv2.imshow("frame", frame) # shows image 
    if cv2.waitKey(20) & 0xFF == 27: # allows keyboard press to close window with the escape key
        break

# when finished, release the capture
cap.release() 
cv2.destroyAllWindows()

In [ ]:
# Establish a connection to the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    ret, frame = cap.read()
   
    # Cut down frame to 250x250pxqqqqqq
    frame = frame[120:120+250,200:200+250, :]
    
    # Collect anchors 
    if cv2.waitKey(1) & 0XFF == ord("a"):
        # Create the unique file path 
        imgname = os.path.join(ANC_PATH, "{}.jpg".format(uuid.uuid1()))
        # Write out anchor image
        cv2.imwrite(imgname, frame)
    
    # Collect positives
    if cv2.waitKey(1) & 0XFF == ord("p"):
        # Create the unique file path 
        imgname = os.path.join(POS_PATH, "{}.jpg".format(uuid.uuid1()))
        # Write out positive image
        cv2.imwrite(imgname, frame)
    
    # Show image back to screen
    cv2.imshow("Image Collection", frame)
    
    # Breaking gracefully
    if cv2.waitKey(1) & 0XFF == ord("q"):
        break
        
# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()

# 3. Load and Pre-process Images

3.1 Get Image Directories

In [17]:
# loads first 300 images in each of the directories
anchor = tf.data.Dataset.list_files(ANC_PATH+"/*.jpg").take(300)
positive = tf.data.Dataset.list_files(POS_PATH+"/*.jpg").take(300) 
negative = tf.data.Dataset.list_files(NEG_PATH+"/*.jpg").take(300)

InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: data\\anchor/*.jpg'

3.2 Preprocessing - Scale and Resize

In [ ]:
def preprocess(file_path):
    # reading img
    byte_img = tf.io.read_file(file_path)
    # using tf decode image to load it in
    img = tf.io.decode_jpeg(byte_img)
    # resizes img
    img = tf.image.resize(img, (100,100))
    # scales image to be between 0 and 1
    img = img / 255.0
    return img

3.3 Create Labelled Dataset

In [10]:
# combines the anchor and positve/negative image. Adds 1.0/0.0 depending if same face
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

NameError: name 'anchor' is not defined

3.4 Build Train and Test Partition

In [ ]:
# puts input_img, validation_img and label into a list
def preprocess_twin(input_img, validation_img, label):
    return (preprocess(input_img), preprocess(validation_img), label)

In [ ]:
# maps our data
data = data.map(preprocess_twin)
# caching our images so we can access them faster
data = data.cache()
# shuffles all our data,
data = data.shuffle(buffer_size=1024)

In [ ]:
#### Training ####
# takes 70% of images for training data
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(32)
# starts preprocessing the next set of images so that we don"t bottle neck our next set images
train_data = train_data.prefetch(8)

In [ ]:
#### Testing ####
# skips first 70% of images
test_data = data.skip(round(len(data)*.7))
# then takes 30% of data left 
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(32)
test_data = test_data.prefetch(8)

# 4. Model Engineering

4.1 Build Embedding Layer

In [ ]:
inp = Input(shape=(100,100,3), name="input_image")

In [ ]:
inp = Input(shape=(100,100,3), name="input_image")

c1 = Conv2D(64, (10,10), activation="relu")(inp)
m1 = MaxPooling2D(64, (2,2), padding="same")(c1)

c2 = Conv2D(128, (7,7), activation="relu")(m1)
m2 = MaxPooling2D(64, (2,2), padding="same")(c2)

c3 = Conv2D(128, (4,4), activation="relu")(m2)
m3 = MaxPooling2D(64, (2,2), padding="same")(c3)

c4 = Conv2D(256, (4,4), activation="relu")(m3)
f1 = Flatten()(c4)
d1 = Dense(4096, activation="sigmoid")(f1)

mod = Model(inputs=[inp], outputs=[d1], name="embedding")
mod.summary()

In [ ]:
def make_embedding():
    inp = Input(shape=(100,100,3), name="input_image")

    #### First block ####
    c1 = Conv2D(64, (10,10), activation="relu")(inp)
    m1 = MaxPooling2D(64, (2,2), padding="same")(c1)

    #### Second Block ####
    c2 = Conv2D(128, (7,7), activation="relu")(m1)
    m2 = MaxPooling2D(64, (2,2), padding="same")(c2)

    #### Third Block ####
    c3 = Conv2D(128, (4,4), activation="relu")(m2)
    m3 = MaxPooling2D(64, (2,2), padding="same")(c3)

    #### Final Embedding Block ####
    c4 = Conv2D(256, (4,4), activation="relu")(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation="sigmoid")(f1)
    return Model(inputs=[inp], outputs=[d1], name="embedding")

In [ ]:
embedding = make_embedding()
embedding.summary()

Distance layer

In [ ]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
        
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

Make model

In [ ]:
def make_siamese_model():
    input_image = Input(name="input_img", shape=(100,100,3))
    validation_image = Input(name="validation_img", shape=(100,100,3))

    
    
    siamese_layer = L1Dist()
    siamese_layer._name = "distance"
    distances = siamese_layer(embedding(input_image), embedding(validation_image))

    classifier = Dense(1, activation="sigmoid")(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=classifier, name="SiameseNetwork")

In [ ]:
siamese_model = make_siamese_model()
siamese_model.summary()

# 5. Training

5.1 Setup Loss and Optimiser

In [ ]:
# the loss will be used later to be able to calculate our loss (1 or 0)
binary_loss = tf.losses.BinaryCrossentropy()

In [ ]:
# improves speed and performance
opt = tf.keras.optimizers.Adam(1e-4) # 0.0001

5.2 Establish Checkpoints

In [ ]:
# defined our checkpoint dir
checkpoint_dir = "./training_checkpoints"
# ensures that all our checkpoints have the prefix of ckpt
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
# saves our the model and optimiser at the time we run the checkpoint class
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

5.3 Build Train Step Function

In [ ]:
@tf.function # compiles our function into a callable TensorFlow graph
def train_step(batch):
    # allows us to capture our gradient from the model, records the operations for automatic differentiation
    with tf.GradientTape() as tape:
        x = batch[:2] # get anchor and positive/negative images
        y = batch[2] # takes the label

        # passes our data through the siamese model to make a prediction
        y_pred = siamese_model(x, training=True)
        # calculates the loss
        loss = binary_loss(y, y_pred)
    # calculates all of the gradients in respect to our loss for all of our trainable variables   
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    # calculate updated weights and apply to siamese model
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))

    return loss



5.4 Build Training Loop

In [ ]:
def train(data, EPOCHS):
    for epoch in range(1, EPOCHS+1):
        print(f"\n Epoch {epoch}/{EPOCHS}")
        progress_bar = tf.keras.utils.Progbar(len(data))

    for idx, batch in enumerate(data):
        train_step(batch)
        progress_bar.update(idx+1)

    if epoch % 10 == 0: 
        checkpoint.save(file_prefix=checkpoint_prefix)

5.5 Train Model

In [ ]:
EPOCHS = 50 # num times we will run through the training data
train(train_data, EPOCHS)

# 6. Evaluate Model

In [ ]:
siamese_model = tf.keras.models.load_model("siamese_model.h5", custom_objects={"L1Dist":L1Dist, "BinaryCrossentropy":tf.losses.BinaryCrossentropy})

6.1 Import Metrics

In [ ]:
# Precision: Computes the precision of the predictions with respect to the labels
# Recall: Computes the recall of the predictions with respect to the labels
from tensorflow.keras.metrics import Precision, Recall 

6.2 Make Predictions

In [ ]:
# unpacks batch
test_input, test_validation, y_true = test_data.as_numpy_iterator().next() # converts our dataset as a numpy equivalent

In [ ]:
# makes prediction
y_pred = siamese_model.predict([test_input, test_validation])
y_pred

In [ ]:
# if prediction is > 0.5, we want our result to add a 1
results = []
for prediction in y_pred:
    if prediction > 0.5:
        results.append(1)
    else:
        results.append(0)
results

In [ ]:
y_true

6.3 Calculating Metrics

In [ ]:
# calculates recall metric
metrics = Recall()
metrics.update_state(y_true, y_pred) # calculating the recall value
metrics = metrics.result().numpy()
metrics

In [ ]:
#  calculates precision metric
metrics = Precision()
metrics.update_state(y_true, y_pred)
metrics = metrics.result().numpy()
metrics

6.4 Visualise Results

In [ ]:
plt.figure(figsize=(18,8))

plt.subplot(1,2,1)
plt.imshow(test_input[0])

plt.subplot(1,2,2)
plt.imshow(test_validation[0])

# 7. Save Model

In [ ]:
#siamese_model.save("siamese_model.h5")

# 8 Real test

In [ ]:
L1Dist

In [ ]:
model = tf.keras.models.load_model("siamese_model.h5", custom_objects={"L1Dist":L1Dist, "BinaryCrossentropy":tf.losses.BinaryCrossentropy})

In [ ]:
def verify(model, detection_threshold, verification_threshold):
    results = []
    for image in os.listdir(os.path.join("application_data", "verification_images")):
        input_img = preprocess(os.path.join("application_data", "input_image", "input_img.jpg"))
        validation_img = preprocess(os.path.join("application_data", "verification_images", image))

        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)

        detection = np.sum(np.array(results) > detection_threshold)

        verification = detection / len(os.listdir(os.path.join("application_data", "verification_images")))
        
        verified = verification > verification_threshold
        return results, verified 

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(3, 250)
cap.set(4, 250)

while True:
    ret, frame = cap.read()

    if cv2.waitKey(1) & 0xFF == ord("c"):
        imname = os.path.join("application_data", "input_image", "input_img.jpg")
        cv2.imwrite(imname, frame)

        results, verified = verify(model, 0.1, 0.1)
        print(verified)

    cv2.imshow("verification", frame) # shows image 
    if cv2.waitKey(20) & 0xFF == 27: # allows keyboard press to close window with the escape key
        break

cap.release() 
cv2.destroyAllWindows()

# 9. Fix issue

In [ ]:
model = tf.keras.models.load_model("siamese_model.h5", custom_objects={"L1Dist":L1Dist, "BinaryCrossentropy":tf.losses.BinaryCrossentropy})

9.1 Change images to same as test values

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
INP_PATH = os.path.join("application_data", "input_image", "files")
VER_PATH = os.path.join("application_data", "verification_images")
NEG_PATH = os.path.join("application_data", "negative_images")

In [ ]:
input_image = tf.data.Dataset.list_files(INP_PATH+"/*.jpg").take(300)
validation_images = tf.data.Dataset.list_files(VER_PATH+"/*.jpg").take(300)
negative_images = tf.data.Dataset.list_files(NEG_PATH+"/*.jpg").take(300)

In [ ]:
def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Preprocessing steps - resizing the image to be 100x100x3
    img = tf.image.resize(img, (100,100))
    # Scale image to be between 0 and 1 
    img = img / 255.0
    
    # Return image
    return img

In [ ]:
pos_data = tf.data.Dataset.zip((input_image, validation_images, tf.data.Dataset.from_tensor_slices(tf.ones(len(input_image)))))
neg_data = tf.data.Dataset.zip((input_image, negative_images, tf.data.Dataset.from_tensor_slices(tf.zeros(len(input_image))))) 

#data = image_dataset_from_directory(os.path.join("application_data", "input_image"), batch_size=1, image_size=(250, 250))
data = pos_data.concatenate(neg_data)

In [ ]:
def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [ ]:
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=1024)

In [ ]:
real_data = data.take(round(len(data)*0.3))
real_data = real_data.batch(32)
real_data = real_data.prefetch(8)

In [ ]:
real_input, real_validation, real_label = real_data.as_numpy_iterator().next()

In [ ]:
y_pred = model.predict([real_input, real_validation])
y_pred

In [ ]:
# if prediction is > 0.5, we want our result to add a 1
results = []
for prediction in y_pred:
    if prediction > 0.5:
        results.append(1)
    else:
        results.append(0)
results

In [ ]:
real_label

In [ ]:
# Creating a metric object 
m = Recall()

# Calculating the recall value 
m.update_state(y_true, y_pred)

# Return Recall Result
m.result().numpy()

In [ ]:
#from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
#real_data = image_dataset_from_directory(os.path.join("application_data", "input_image"), batch_size=1, image_size=(250, 250))

9.2 Implement into live design

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
def data_change():
    # takes path for files
    INP_PATH = os.path.join("application_data", "input_image", "files")
    VER_PATH = os.path.join("application_data", "verification_images")
    NEG_PATH = os.path.join("application_data", "negative_images")

    # takes 50 images and puts them into dataset
    input_image = tf.data.Dataset.list_files(INP_PATH+"/*.jpg").take(32)
    validation_images = tf.data.Dataset.list_files(VER_PATH+"/*.jpg").take(32)
    negative_images = tf.data.Dataset.list_files(NEG_PATH+"/*.jpg").take(32)

    # ensures data is readable for the map function
    #pos_data = image_dataset_from_directory(os.path.join("application_data", "input_image"), batch_size=1, image_size=(250, 250))
    
    # creates our whole data (input_image, verification_image, label)
    pos_data = tf.data.Dataset.zip((input_image, validation_images, tf.data.Dataset.from_tensor_slices(tf.ones(len(input_image)))))
    neg_data = tf.data.Dataset.zip((input_image, negative_images, tf.data.Dataset.from_tensor_slices(tf.ones(len(input_image)))))
    data = pos_data.concatenate(neg_data)

    data = data.map(preprocess_twin)
    data = data.cache()
    #data = data.shuffle(buffer_size=1024)
    data = data.take(32)
    data = data.batch(32)
    data = data.prefetch(8)

    global real_input
    global real_validation
    real_input, real_validation, null = data.as_numpy_iterator().next()
    # takes images from the dataset
    #return real_input, real_validation

In [ ]:
def verify(model, detection_threshold, verification_threshold):
    # changes data
    data_change()
    # predicts results
    results = []
    results2 = []
    y_pred = model.predict([real_input, real_validation])

    for prediction in y_pred:
        if prediction > 0.1:
            results2.append(1)
        else:
            results2.append(0)
    print(results2)
    results.append(y_pred)


    
    # Detection Threshold: Metric above which a prediciton is considered positive 
    detection = np.sum(np.array(results) > detection_threshold)
    
    # Verification Threshold: Proportion of positive predictions / total positive samples 
    verification = detection / len(os.listdir(os.path.join("application_data", "verification_images"))) 
    verified = verification > verification_threshold
    
    return results, verified

In [ ]:
import uuid

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]
    
    cv2.imshow("Verification", frame)
    
    # Verification trigger
    if cv2.waitKey(1) & 0xFF == ord("v"):
        # removes existing images
        for image in os.listdir(os.path.join("application_data", "input_image", "files")):
            image_remove = os.path.join("application_data", "input_image", "files", image)
            os.remove(image_remove)
        
        # Save input image to application_data/input_image folder 
        image_path = os.path.join("application_data", "input_image", "files")
        for i in range(32):
            imgname = os.path.join(image_path, "{}.jpg".format(uuid.uuid1()))
            cv2.imwrite(imgname, frame)
        # Run verification
        results, verified = verify(model, 0.5, 0.5)
        print(verified)
        break

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
data_change()
y_pred = model.predict([real_input, real_validation])
y_pred

In [ ]:
results = []
for prediction in y_pred:
    if prediction > 0.5:
        results.append(1)
    else:
        results.append(0)
results

# Fix issue 2

In [1]:
import cv2
import os
import random
import numpy as np 
from PIL import Image
from matplotlib import pyplot as plt
import uuid
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall 


2022-11-02 11:07:56.202889: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-02 11:07:58.570633: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/lukas/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-11-02 11:07:58.570664: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-02 11:07:58.612350: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already b

In [2]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
        
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [3]:
siamese_model = tf.keras.models.load_model("siamese_model.h5", custom_objects={"L1Dist":L1Dist, "BinaryCrossentropy":tf.losses.BinaryCrossentropy})

2022-11-02 11:08:08.431796: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/lukas/.local/lib/python3.10/site-packages/cv2/../../lib64:
2022-11-02 11:08:08.431861: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-02 11:08:08.431898: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lukas-HP-Pavilion-Notebook): /proc/driver/nvidia/version does not exist
2022-11-02 11:08:08.432273: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-02 11

2022-11-02 11:08:09.472545: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 150994944 exceeds 10% of free system memory.


In [17]:
POS_PATH = os.path.join("application_data", "input_image")
NEG_PATH = os.path.join("application_data", "negative_images")
ANC_PATH = os.path.join("application_data", "verification_images")

In [18]:
# Establish a connection to the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    ret, frame = cap.read()
    # Cut down frame to 250x250px
    frame = frame[120:120+250,200:200+250, :]
    
    # Collect anchors 
    if cv2.waitKey(1) & 0XFF == ord('a'):
        for image in os.listdir(os.path.join("application_data", "input_image")):
            image_remove = os.path.join("application_data", "input_image", image)
            os.remove(image_remove)
        for i in range(300):
            # Create the unique file path 
            imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
            # Write out anchor image
            cv2.imwrite(imgname, frame)
        break
    

    cv2.imshow('Image Collection', frame)
    
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()

In [19]:
anchor = tf.data.Dataset.list_files(ANC_PATH+"/*.jpg").take(300)
positive = tf.data.Dataset.list_files(POS_PATH+"/*.jpg").take(300)
negative = tf.data.Dataset.list_files(NEG_PATH+"/*.jpg").take(300)

In [20]:
def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Preprocessing steps - resizing the image to be 100x100x3
    img = tf.image.resize(img, (100,100))
    # Scale image to be between 0 and 1 
    img = img / 255.0
    
    # Return image
    return img

In [21]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
data = positives.concatenate(negatives)

In [22]:
def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [23]:
data = data.map(preprocess_twin)
data = data.cache()

In [24]:
# Testing partition
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(32)
test_data = test_data.prefetch(8)

In [25]:
test_input, test_validation, y_true = test_data.as_numpy_iterator().next()

2022-11-02 11:09:02.500319: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [26]:
y_pred = siamese_model.predict([test_input, test_validation])
y_pred

1/1 [==============================] - 4s 4s/step


array([[0.00065651],
       [0.50116116],
       [0.00067683],
       [0.50116116],
       [0.50116116],
       [0.50116116],
       [0.50116116],
       [0.50116116],
       [0.50116116],
       [0.50116116],
       [0.50116116],
       [0.50116116],
       [0.50116116],
       [0.50116116],
       [0.50116116],
       [0.50116116]], dtype=float32)

In [27]:
results = []
for prediction in y_pred:
    if prediction < 0.6:
        results.append(1)
    else:
        results.append(0)
results

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [28]:
# Creating a metric object 
m = Recall()
# Calculating the recall value 
m.update_state(y_true, y_pred)
# Return Recall Result
m.result().numpy()

0.875

In [29]:
if m.result().numpy() >= 0.875:
    print("True")
else:
    print("False")

True
